## Imports

In [1]:
from nltk.corpus import stopwords
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

2025-05-26 19:47:46.931294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748288867.151500      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748288867.215614      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data loading

In [28]:
df = pd.read_csv("/kaggle/input/dsaa-6100-movie-review-sentiment-classification/movie_reviews/movie_reviews.csv")
df.head()

,text,label
0,If you havent seen this movie than you need to...,1
1,but Cinderella gets my vote not only for the w...,0
2,This movie is pretty cheesy but I do give it c...,1
3,I have not seen a Van Damme flick for a while ...,1
4,This is a sleeper It defines Nicholas Cage The...,1


In [29]:
dataset = Dataset.from_pandas(df)

## Pretrained  distilbert-base-uncased-finetuned-mnli-amazon-query-shopping

In [30]:
from transformers import pipeline


pipe = pipeline("text-classification", model="LiYuan/amazon-review-sentiment-analysis", truncation=True)

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [31]:
reviews = ["Best film", "worst film"]
pipe(reviews)

[{'label': '5 stars', 'score': 0.951850414276123},
 {'label': '1 star', 'score': 0.918562650680542}]

## Submit

In [32]:
test = pd.read_csv("/kaggle/input/dsaa-6100-movie-review-sentiment-classification/test_data.csv")
test.head()

,Id,text
0,0,What can possibly said about this movie other ...
1,1,I dont care how many bad reviews purple rain g...
2,2,Ken Russell directed this weird Not very ero...
3,3,This is a great movie from the lost age of rea...
4,4,I have a problem with the movie snobs who cons...


In [33]:
test.shape

(10000, 2)

In [34]:
test.isna().sum()

Id      0
text    0
dtype: int64

In [35]:
results = pipe(test["text"].tolist())

In [36]:
results[:5]

[{'label': '1 star', 'score': 0.8556066751480103},
 {'label': '5 stars', 'score': 0.807982325553894},
 {'label': '1 star', 'score': 0.422488808631897},
 {'label': '5 stars', 'score': 0.8241698741912842},
 {'label': '4 stars', 'score': 0.42969003319740295}]

In [39]:
maps = {
    "5 stars": 1,
    "4 stars": 1,
    "3 stars": 0,
    "2 stars": 0,
    "1 star": 0,
}

pd.DataFrame({
    "Id": list(range(len(results))),
    "Category": [maps[x["label"]] for x in results]
}).to_csv("distilbert_pretrained_amazon.csv", index=False)